#1.Linear, RandomForest, LightGBM 3가지 모델 비교

In [1]:
# ============================================================
# 📊 아파트 전세가 예측 (Linear, RandomForest, LightGBM 비교)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# 1️⃣ 데이터 불러오기
file_path = "아파트(전세)_서울마포구_갱신미사용만.csv"
df = pd.read_csv(file_path)

# 2️⃣ 불필요한 공백 제거 및 확인
df = df.copy()
df.columns = df.columns.str.strip()

# 3️⃣ 피처(X), 타깃(y) 설정
X = df.drop(columns=["보증금(만원)"])
y = df["보증금(만원)"]

# 4️⃣ 범주형 컬럼 인코딩
cat_cols = X.select_dtypes(include=["object"]).columns
encoder = LabelEncoder()
for col in cat_cols:
    X[col] = encoder.fit_transform(X[col])

# 5️⃣ 데이터 분리 (Train 80%, Test 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6️⃣ 모델 정의
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
}

# 7️⃣ 결과 저장용 DataFrame
results = []

# 8️⃣ 각 모델별 학습 및 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)

    results.append({
        "Model": name,
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 4)
    })

# 9️⃣ 결과 출력
results_df = pd.DataFrame(results)
print("✅ 모델별 성능 비교 (MAE / RMSE / R²)")
print(results_df)

# 10️⃣ 결과 저장 (선택)
results_df.to_csv("모델_비교결과.csv", index=False)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000275 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 564
[LightGBM] [Info] Number of data points in the train set: 4111, number of used features: 8
[LightGBM] [Info] Start training from score 65733.477986
✅ 모델별 성능 비교 (MAE / RMSE / R²)
               Model       MAE      RMSE      R²
0  Linear Regression  12556.67  17140.56  0.6340
1      Random Forest   5564.47   9037.88  0.8982
2           LightGBM   5699.34   9186.61  0.8949


# 2.RandomForest모델로 저평가 항목 선별

In [6]:
# ============================================================
# 📈 Random Forest 예측값 - 실제값 (차액 기준) Top10 추출 + 엑셀 저장
#  - 차액 정의: 차액(예측-실제)
#  - 병합 없이 테스트셋 원본에 컬럼 추가 → _x, _y KeyError 방지
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

# 1) 데이터 불러오기
file_path = "아파트(전세)_서울마포구_갱신미사용만.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()

# 2) X, y 분리
X = df.drop(columns=["보증금(만원)"])
y = df["보증금(만원)"]

# 3) 범주형 인코딩 (LabelEncoder)
cat_cols = X.select_dtypes(include=["object"]).columns
for col in cat_cols:
    X[col] = X[col].astype(str).str.strip()
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# 4) Train/Test 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 5) 모델 학습 (Random Forest)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)

# 6) 예측
preds = rf.predict(X_test)

# 7) 결과 테이블 생성 (테스트셋 '원본 행' 기반)
#    - y_test.index로 원본 df에서 해당 행을 꺼내 복사
orig_test = df.loc[y_test.index].copy()
result = orig_test.copy()
result["예측값(보증금)"] = preds
result["차액(예측-실제)"] = result["예측값(보증금)"] - result["보증금(만원)"]

# 8) 차액(예측-실제) 기준 Top10 (고평가 항목)
top10_over = result.sort_values("차액(예측-실제)", ascending=False).head(10)

# 9) 엑셀 저장
output_path = "RandomForest_고평가_Top10.xlsx"
top10_over.to_excel(output_path, index=False)

# 10) 확인용 출력
view_cols = [
    "단지명", "시군구", "전용면적(㎡)", "층", "건축년도", "계약년월", "계약일",
    "보증금(만원)", "예측값(보증금)", "차액(예측-실제)"
]
print("✅ 예측값 - 실제값 기준 (고평가) Top10")
print(top10_over[view_cols].reset_index(drop=True))
print(f"\n📁 저장 위치: {output_path}")


✅ 예측값 - 실제값 기준 (고평가) Top10
                    단지명            시군구   전용면적(㎡)   층  건축년도    계약년월  계약일  \
0          마포래미안푸르지오1단지  서울특별시 마포구 아현동   84.8919  23  2014  202410    7   
1            상암월드컵파크5단지  서울특별시 마포구 상암동  104.6900  18  2005  202505   14   
2            상암월드컵파크4단지  서울특별시 마포구 상암동   84.7000   9  2006  202504   25   
3               래미안웰스트림  서울특별시 마포구 현석동   84.9800  25  2016  202507   30   
4           함성월드빌3차301동  서울특별시 마포구 망원동   84.8000   8  2005  202412    7   
5               래미안공덕5차  서울특별시 마포구 공덕동   59.9590  15  2011  202411    6   
6            마포공덕파크팰리스Ⅱ  서울특별시 마포구 공덕동  119.0700  11  2005  202411   30   
7                    두영  서울특별시 마포구 용강동  130.8600   4  2003  202507   16   
8  마포자이더센트리지(102동~112동)  서울특별시 마포구 염리동  119.5740  25  2022  202410   12   
9           이편한세상마포리버파크  서울특별시 마포구 용강동   84.9800   1  2016  202508   19   

   보증금(만원)    예측값(보증금)  차액(예측-실제)  
0    20000   89732.500  69732.500  
1    10000   77678.000  67678.000  
2    10000   68529.500  58529.500  
3  

#3.'마포래미안푸르지오1단지'의 전세금 정렬 비교 (이상치 확인목적)

In [7]:
# ============================================================
# 📊 '마포래미안푸르지오1단지' 전세금 기준 오름차순 정렬 + 엑셀 저장
# ============================================================

import pandas as pd

# 1️⃣ CSV 파일 불러오기
file_path = "아파트(전세)_서울마포구_갱신미사용만.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()   # 혹시 모를 공백 제거

# 2️⃣ '마포래미안푸르지오1단지'만 필터링
target_name = "마포래미안푸르지오1단지"
df_target = df[df["단지명"].astype(str).str.strip() == target_name].copy()

# 3️⃣ 전세금(보증금) 기준 오름차순 정렬
df_sorted = df_target.sort_values(by="보증금(만원)", ascending=True).reset_index(drop=True)

# 4️⃣ 엑셀 파일로 저장
output_path = "마포래미안푸르지오1단지_전세금_오름차순.xlsx"
df_sorted.to_excel(output_path, index=False)

# 5️⃣ 결과 확인용 출력
print("✅ '마포래미안푸르지오1단지' 전세금 기준 오름차순 정렬 완료!")
print(f"📁 저장 위치: {output_path}\n")
print(df_sorted.head(10))  # 상위 10건 미리보기


✅ '마포래미안푸르지오1단지' 전세금 기준 오름차순 정렬 완료!
📁 저장 위치: 마포래미안푸르지오1단지_전세금_오름차순.xlsx

             시군구           단지명  전용면적(㎡)    계약년월  계약일  보증금(만원)   층  건축년도 계약구분
0  서울특별시 마포구 아현동  마포래미안푸르지오1단지  36.9796  202504   17     6231   5  2014   갱신
1  서울특별시 마포구 아현동  마포래미안푸르지오1단지  36.9796  202509    9     6474   2  2014   갱신
2  서울특별시 마포구 아현동  마포래미안푸르지오1단지  36.9796  202509    8     6474   4  2014    -
3  서울특별시 마포구 아현동  마포래미안푸르지오1단지  41.9838  202501   31     7049  15  2014   갱신
4  서울특별시 마포구 아현동  마포래미안푸르지오1단지  41.9838  202509    9     7324   3  2014   갱신
5  서울특별시 마포구 아현동  마포래미안푸르지오1단지  84.8919  202410    7    20000  23  2014    -
6  서울특별시 마포구 아현동  마포래미안푸르지오1단지  59.9236  202502   25    61950   8  2014    -
7  서울특별시 마포구 아현동  마포래미안푸르지오1단지  59.9236  202507   12    65000  18  2014   신규
8  서울특별시 마포구 아현동  마포래미안푸르지오1단지  59.9452  202410   12    66150   2  2014   갱신
9  서울특별시 마포구 아현동  마포래미안푸르지오1단지  59.9236  202410    5    67000  16  2014   갱신


In [10]:
%pip install xlsxwriter

#4.모든 이상치 탐지

In [11]:
# ============================================================
# 🧪 전세 데이터 아웃라이어(이상치) 탐지 스크립트
#  - 대상 파일: 아파트(전세)_서울마포구_갱신미사용만.csv
#  - 방법1: IQR (Q1-1.5*IQR, Q3+1.5*IQR)
#  - 방법2: Z-Score (|z| >= 3)
#  - 방법3: IsolationForest (contamination 비율로 조절)
#  - 출력: 콘솔 요약 + Excel로 결과 저장
# ============================================================

import pandas as pd
import numpy as np

from scipy.stats import zscore
from sklearn.ensemble import IsolationForest

# 0) 설정
FILE_PATH = "아파트(전세)_서울마포구_갱신미사용만.csv"
OUTPUT_XLSX = "outliers_전세_마포.xlsx"

# (선택) 핵심 컬럼 지정: 없으면 자동으로 수치형 전체 사용
KEY_NUMERIC_COLS = [
    "보증금(만원)", "전용면적(㎡)", "층", "건축년도", "계약년월", "계약일"
]

# 1) 데이터 로드 & 정리
df = pd.read_csv(FILE_PATH)
df.columns = df.columns.str.strip()
df = df.copy()

# 수치형 컬럼 자동 식별
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# KEY_NUMERIC_COLS가 실제 존재하는 컬럼으로만 필터링
target_cols = [c for c in KEY_NUMERIC_COLS if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
if not target_cols:
    # 지정한 핵심 컬럼이 없거나 비수치형이면, 수치형 전체로 대체
    target_cols = numeric_cols

# ============== 방법1: IQR 기준 ==============
def iqr_outlier_mask(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return (series < lower) | (series > upper), lower, upper

iqr_details = {}
iqr_masks = []
for col in target_cols:
    mask, lo, hi = iqr_outlier_mask(df[col].astype(float), k=1.5)
    iqr_details[col] = {
        "Q1": df[col].quantile(0.25),
        "Q3": df[col].quantile(0.75),
        "IQR": df[col].quantile(0.75) - df[col].quantile(0.25),
        "LowerBound": lo, "UpperBound": hi,
        "OutlierCount": int(mask.sum()),
        "OutlierRate(%)": round(mask.mean()*100, 2)
    }
    iqr_masks.append(mask)

# 컬럼별 IQR 요약표
iqr_summary = pd.DataFrame(iqr_details).T.reset_index().rename(columns={"index": "column"})

# IQR 기준 통합 마스크(하나라도 이상치면 True)
iqr_any_mask = np.logical_or.reduce(iqr_masks) if iqr_masks else pd.Series(False, index=df.index)
iqr_outliers_df = df[iqr_any_mask].copy()

# ============== 방법2: Z-Score 기준 (|z| >= 3) ==============
zscore_df = df[target_cols].apply(lambda s: zscore(s.astype(float), nan_policy='omit'))
zscore_mask = (zscore_df.abs() >= 3).any(axis=1)
z_outliers_df = df[zscore_mask].copy()

# 컬럼별 z 이상치 수 요약
z_col_counts = (zscore_df.abs() >= 3).sum().rename("OutlierCount").to_frame()
z_col_counts["OutlierRate(%)"] = ((zscore_df.abs() >= 3).mean()*100).round(2)
z_summary = z_col_counts.reset_index().rename(columns={"index": "column"})

# ============== 방법3: IsolationForest (선택) ==============
# 데이터 규모/분포 따라 contamination 조절(예: 0.01 = 1%)
use_isoforest = True
iso_outliers_df = pd.DataFrame()
iso_summary = pd.DataFrame(columns=["Model", "Contamination", "OutlierCount", "OutlierRate(%)"])

if use_isoforest and len(target_cols) > 0:
    X = df[target_cols].astype(float).fillna(df[target_cols].median(numeric_only=True))
    iso = IsolationForest(
        n_estimators=300,
        contamination=0.01,
        random_state=42,
        n_jobs=-1
    )
    preds = iso.fit_predict(X)   # 정상 1, 이상치 -1
    iso_mask = (preds == -1)
    iso_outliers_df = df[iso_mask].copy()
    iso_summary = pd.DataFrame([{
        "Model": "IsolationForest",
        "Contamination": 0.01,
        "OutlierCount": int(iso_mask.sum()),
        "OutlierRate(%)": round(iso_mask.mean()*100, 2)
    }])

# ============== 전체 요약 출력 ==============
print("✅ 대상 수치 컬럼:", target_cols)
print("\n[방법1: IQR 요약]")
print(iqr_summary.sort_values("OutlierCount", ascending=False).reset_index(drop=True))

print("\n[방법2: Z-Score 요약 (|z| >= 3)]")
print(z_summary.sort_values("OutlierCount", ascending=False).reset_index(drop=True))

if not iso_summary.empty:
    print("\n[방법3: IsolationForest 요약]")
    print(iso_summary)

print("\nIQR 기준 이상치 행 수:", len(iqr_outliers_df))
print("Z-Score 기준 이상치 행 수:", len(z_outliers_df))
if not iso_outliers_df.empty:
    print("IsolationForest 기준 이상치 행 수:", len(iso_outliers_df))

# ============== 엑셀로 저장(여러 시트) ==============
with pd.ExcelWriter(OUTPUT_XLSX, engine="xlsxwriter") as writer:
    df.to_excel(writer, index=False, sheet_name="원본데이터")
    iqr_summary.to_excel(writer, index=False, sheet_name="IQR_요약")
    z_summary.to_excel(writer, index=False, sheet_name="Z_요약")
    iqr_outliers_df.to_excel(writer, index=False, sheet_name="IQR_이상치행")
    z_outliers_df.to_excel(writer, index=False, sheet_name="Z_이상치행")
    if not iso_outliers_df.empty:
        iso_summary.to_excel(writer, index=False, sheet_name="IF_요약")
        iso_outliers_df.to_excel(writer, index=False, sheet_name="IF_이상치행")

print(f"\n📁 저장 완료: {OUTPUT_XLSX}")


✅ 대상 수치 컬럼: ['보증금(만원)', '전용면적(㎡)', '층', '건축년도', '계약년월', '계약일']

[방법1: IQR 요약]
    column         Q1         Q3       IQR  LowerBound  UpperBound  \
0  전용면적(㎡)      59.73      84.94     25.21      21.915     122.755   
1  보증금(만원)   46300.00   82000.00  35700.00   -7250.000  135550.000   
2        층       5.00      13.00      8.00      -7.000      25.000   
3     건축년도    1999.00    2014.00     15.00    1976.500    2036.500   
4     계약년월  202412.00  202506.00     94.00  202271.000  202647.000   
5      계약일       9.00      23.00     14.00     -12.000      44.000   

   OutlierCount  OutlierRate(%)  
0         194.0            3.78  
1          82.0            1.60  
2          68.0            1.32  
3           9.0            0.18  
4           0.0            0.00  
5           0.0            0.00  

[방법2: Z-Score 요약 (|z| >= 3)]
    column  OutlierCount  OutlierRate(%)
0        층            47            0.91
1  보증금(만원)            42            0.82
2  전용면적(㎡)            38            0.74

#5.탐지된 이상치건 총 290건 모두 삭제 후 재 학습 및 전과 비교

In [13]:
# ============================================================
# 📈 이상치 제거 후 Random Forest 회귀 모델 학습 + 성능 평가 + 차액 Top10
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1️⃣ 데이터 불러오기
file_path = "아파트(전세)_서울마포구_갱신미사용만.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()

# ------------------------------------------------------------
# 2️⃣ IQR 기반 이상치 제거 (주요 수치형 컬럼)
# ------------------------------------------------------------
numeric_cols = ['보증금(만원)', '전용면적(㎡)', '층', '건축년도', '계약년월', '계약일']

def iqr_filter(df, cols, k=1.5):
    mask = pd.Series(False, index=df.index)
    for col in cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - k * iqr
        upper = q3 + k * iqr
        mask |= (df[col] < lower) | (df[col] > upper)
    print(f"🧹 제거된 이상치 개수: {mask.sum()} / 전체 {len(df)}개 ({round(mask.mean()*100,2)}%)")
    return df[~mask].copy()

df_clean = iqr_filter(df, numeric_cols)

# ------------------------------------------------------------
# 3️⃣ 학습 데이터 준비
# ------------------------------------------------------------
X = df_clean.drop(columns=["보증금(만원)"])
y = df_clean["보증금(만원)"]

# 범주형 인코딩
cat_cols = X.select_dtypes(include=["object"]).columns
for col in cat_cols:
    X[col] = X[col].astype(str).str.strip()
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ------------------------------------------------------------
# 4️⃣ Random Forest 회귀 모델 학습
# ------------------------------------------------------------
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
preds = rf.predict(X_test)

# ------------------------------------------------------------
# 5️⃣ 모델 평가 (MAE, RMSE, R²)
# ------------------------------------------------------------
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
r2 = r2_score(y_test, preds)

print("\n📊 [모델 학습 성능 평가 결과]")
print(f"MAE (평균절대오차): {mae:,.2f}")
print(f"RMSE(평균제곱근오차): {rmse:,.2f}")
print(f"R² (결정계수): {r2:.4f}")

# ------------------------------------------------------------
# 6️⃣ 예측값 - 실제값 = 차액 계산
# ------------------------------------------------------------
orig_test = df_clean.loc[y_test.index].copy()
orig_test["예측값(보증금)"] = preds
orig_test["차액(예측-실제)"] = orig_test["예측값(보증금)"] - orig_test["보증금(만원)"]

# ------------------------------------------------------------
# 7️⃣ 차액 기준 Top10 (고평가 항목)
# ------------------------------------------------------------
top10_over = orig_test.sort_values(by="차액(예측-실제)", ascending=False).head(10)

# ------------------------------------------------------------
# 8️⃣ 엑셀 저장
# ------------------------------------------------------------
output_path = "RandomForest_고평가_Top10_이상치제거후.xlsx"
top10_over.to_excel(output_path, index=False)

# ------------------------------------------------------------
# 9️⃣ 결과 출력
# ------------------------------------------------------------
print("\n✅ 이상치 제거 후 고평가 Top10 항목")
cols = ["단지명", "시군구", "전용면적(㎡)", "층", "건축년도", "계약년월", "보증금(만원)", "예측값(보증금)", "차액(예측-실제)"]
print(top10_over[cols].reset_index(drop=True))
print(f"\n📁 결과 저장 위치: {output_path}")


🧹 제거된 이상치 개수: 290 / 전체 5139개 (5.64%)

📊 [모델 학습 성능 평가 결과]
MAE (평균절대오차): 5,812.34
RMSE(평균제곱근오차): 9,682.35
R² (결정계수): 0.8647

✅ 이상치 제거 후 고평가 Top10 항목
            단지명            시군구   전용면적(㎡)   층  건축년도    계약년월  보증금(만원)  \
0       래미안웰스트림  서울특별시 마포구 현석동   84.9800  22  2016  202410    29000   
1  마포래미안푸르지오1단지  서울특별시 마포구 아현동   84.8919  23  2014  202410    20000   
2    상암월드컵파크5단지  서울특별시 마포구 상암동  104.6900  18  2005  202505    10000   
3        대우미래사랑  서울특별시 마포구 서교동   84.9600   5  2005  202505     8000   
4   함성월드빌3차301동  서울특별시 마포구 망원동   84.8000   8  2005  202412    10000   
5         예인하우스  서울특별시 마포구 상암동   81.2000   5  2012  202503    33000   
6         예인하우스  서울특별시 마포구 상암동   81.2000   5  2012  202503    33000   
7      공덕1삼성래미안  서울특별시 마포구 공덕동  114.8800   1  1999  202508    40000   
8         예인하우스  서울특별시 마포구 상암동   71.9500   6  2012  202508    38000   
9       신촌숲아이파크  서울특별시 마포구 신수동   84.6650  16  2019  202504    30000   

     예측값(보증금)  차액(예측-실제)  
0  101936.665  72936.665  
1   88245.250  68

#6.이상치 제거한 데이터(train+test)범위내에서 전세금 저평가항목 선별 및 평균차액과 비교.

In [14]:
# ============================================================
# 📊 Random Forest 결과에서 새차액 및 평균차액대비저평가율 계산
# ============================================================

import pandas as pd

# 1️⃣ 기존 예측 결과 파일 불러오기
# (이전 단계에서 생성된 'RandomForest_고평가_Top10_이상치제거후.xlsx' 파일이 아닌,
# 전체 테스트셋 결과(예측값/실제값 포함) 파일을 불러야 함)
file_path = "RandomForest_고평가_Top10_이상치제거후.xlsx"
df = pd.read_excel(file_path)

# 2️⃣ MAE(평균절대오차) 설정값
MAE_VALUE = 5812.34

# 3️⃣ 새차액 및 평균차액대비저평가율 계산
df["새차액"] = df["예측값(보증금)"] - df["보증금(만원)"] - MAE_VALUE
df["평균차액대비저평가율"] = df["새차액"] / MAE_VALUE

# 4️⃣ 새차액 기준 내림차순 Top10 추출
top10_new = df.sort_values(by="새차액", ascending=False).head(10)

# 5️⃣ 결과 출력
cols = ["단지명", "시군구", "전용면적(㎡)", "층", "건축년도",
        "계약년월", "보증금(만원)", "예측값(보증금)",
        "새차액", "평균차액대비저평가율"]
print("✅ 새차액 기준 내림차순 Top10 (평균차액대비저평가율 포함)\n")
print(top10_new[cols].reset_index(drop=True))

# 6️⃣ 엑셀로 저장
output_path = "RandomForest_새차액_저평가율_Top10.xlsx"
top10_new.to_excel(output_path, index=False)

print(f"\n📁 저장 완료: {output_path}")


✅ 새차액 기준 내림차순 Top10 (평균차액대비저평가율 포함)

            단지명            시군구   전용면적(㎡)   층  건축년도    계약년월  보증금(만원)  \
0       래미안웰스트림  서울특별시 마포구 현석동   84.9800  22  2016  202410    29000   
1  마포래미안푸르지오1단지  서울특별시 마포구 아현동   84.8919  23  2014  202410    20000   
2    상암월드컵파크5단지  서울특별시 마포구 상암동  104.6900  18  2005  202505    10000   
3        대우미래사랑  서울특별시 마포구 서교동   84.9600   5  2005  202505     8000   
4   함성월드빌3차301동  서울특별시 마포구 망원동   84.8000   8  2005  202412    10000   
5         예인하우스  서울특별시 마포구 상암동   81.2000   5  2012  202503    33000   
6         예인하우스  서울특별시 마포구 상암동   81.2000   5  2012  202503    33000   
7      공덕1삼성래미안  서울특별시 마포구 공덕동  114.8800   1  1999  202508    40000   
8         예인하우스  서울특별시 마포구 상암동   71.9500   6  2012  202508    38000   
9       신촌숲아이파크  서울특별시 마포구 신수동   84.6650  16  2019  202504    30000   

     예측값(보증금)        새차액  평균차액대비저평가율  
0  101936.665  67124.325   11.548589  
1   88245.250  62432.910   10.741441  
2   77905.500  62093.160   10.682988  
3   64704.050  50891.710 